In [121]:
import pandas as pd
import yfinance as yf

Los datos de los 5 factores Fama-French se tomarán para los años 2020 a 2024, es decir del 2020-01-01 al 2024-12-31.

In [135]:
start_date = "2015-01-01"
end_date = "2025-11-04"

In [136]:
#Cargando los 10 ETF Sectoriales de S&P500
#XLC
etfs_name = ['XLF', 'XLK', 'XLV', 'XLE', 'XLI', 'XLY', 'XLP', 'XLU', 'XLRE', 'XLB']

efts_data = []

for name in etfs_name:
    
    data = yf.download(name, start=start_date, end=end_date)
    
    if data.empty:
            print(f"No hay datos para {name}")
    else:
        data.index = data.index.normalize()

        #Modificando para obtener la data mensual
        monthly_data = data[['Close']].resample('M').last()
        monthly_data[name] = monthly_data['Close'].pct_change()
        efts_data.append(monthly_data[name])

C:\Users\crdie\AppData\Local\Temp\ipykernel_23276\530410504.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(name, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
C:\Users\crdie\AppData\Local\Temp\ipykernel_23276\530410504.py:17: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_data = data[['Close']].resample('M').last()
C:\Users\crdie\AppData\Local\Temp\ipykernel_23276\530410504.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(name, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
C:\Users\crdie\AppData\Local\Temp\ipykernel_23276\530410504.py:17: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_data = data[['Close']].resample('M').last()
C:\Users\crdie\A

In [137]:
#Uniendo todos los datasets en uno solo
asset_returns_df = pd.concat(efts_data, axis=1)
asset_returns_df.index = pd.to_datetime(asset_returns_df.index)

#Modificando el date a formato yyyymm
asset_returns_df.index = asset_returns_df.index.strftime('%Y%m')
asset_returns_df

,XLF,XLK,XLV,XLE,XLI,XLY,XLP,XLU,XLRE,XLB
Date,,,,,,,,,,
201501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201502,0.058236,0.079950,0.042875,0.045929,0.053509,0.085440,0.041441,-0.063948,NaN,0.079682
201503,-0.006160,-0.034356,0.006407,-0.011552,-0.025460,-0.004830,-0.019426,-0.009953,NaN,-0.049048
201504,0.000830,0.027510,-0.010897,0.065739,-0.002510,-0.000530,-0.007591,-0.004726,NaN,0.033620
201505,0.019478,0.018553,0.045042,-0.051887,0.003236,0.013145,0.008683,0.006332,NaN,0.003768
...,...,...,...,...,...,...,...,...,...,...
202507,0.000000,0.037555,-0.032347,0.028299,0.030436,0.018865,-0.014697,0.049106,-0.000241,-0.000911
202508,0.030934,-0.001104,0.053669,0.036464,0.000000,0.046561,0.012534,-0.015758,0.021734,0.051864
202509,0.001062,0.075337,0.017292,-0.003182,0.018796,0.035914,-0.023177,0.041236,0.003346,-0.024217


In [138]:
# Cargando los 5 Fama-French factores

raw_data_ff = pd.read_csv('MonthlyFF4F.csv', skiprows=3)

#Pequeña limpieza de nombre de columnas
ff_cols = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']
fama_french_df = raw_data_ff.rename(columns={raw_data_ff.columns[0]: 'Date'})
fama_french_df.set_index('Date', inplace=True)
fama_french_df[ff_cols] = fama_french_df[ff_cols] / 100
fama_french_df.index = fama_french_df.index.astype(str)
fama_french_df

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
196307,-0.0039,-0.0048,-0.0081,0.0064,-0.0115,0.0027
196308,0.0508,-0.0080,0.0170,0.0040,-0.0038,0.0025
196309,-0.0157,-0.0043,0.0000,-0.0078,0.0015,0.0027
196310,0.0254,-0.0134,-0.0004,0.0279,-0.0225,0.0029
196311,-0.0086,-0.0085,0.0173,-0.0043,0.0227,0.0027
...,...,...,...,...,...,...
202503,-0.0639,-0.0149,0.0290,0.0211,-0.0047,0.0034
202504,-0.0084,-0.0186,-0.0340,-0.0285,-0.0267,0.0035
202505,0.0606,-0.0072,-0.0288,0.0126,0.0251,0.0038


In [141]:
final_df = asset_returns_df.join(fama_french_df,on='Date')

final_df = final_df.dropna()

final_df

,XLF,XLK,XLV,XLE,XLI,XLY,XLP,XLU,XLRE,XLB,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,,,,,,,,,,,
201511,0.019934,0.007102,-0.003224,-0.000147,0.008660,-0.002347,-0.009222,-0.021257,-0.006462,0.009938,0.0057,0.0330,-0.0043,-0.0265,-0.0106,0.0000
201512,-0.023644,-0.020811,0.017229,-0.105170,-0.025724,-0.028431,0.029177,0.021191,0.025327,-0.043401,-0.0215,-0.0295,-0.0257,0.0034,0.0012,0.0001
201601,-0.088544,-0.037124,-0.077190,-0.034980,-0.056971,-0.051817,0.005348,0.049445,-0.063943,-0.107093,-0.0574,-0.0344,0.0208,0.0278,0.0305,0.0001
201602,-0.029005,-0.006547,-0.003611,-0.028002,0.042609,0.004453,0.003152,0.019375,0.004796,0.078153,-0.0007,0.0087,-0.0061,0.0331,0.0194,0.0002
201603,0.071591,0.088217,0.027075,0.101659,0.069735,0.066974,0.047383,0.079765,0.106971,0.075731,0.0695,0.0100,0.0122,0.0068,0.0001,0.0002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202503,-0.041952,-0.082860,-0.015813,0.034932,-0.035630,-0.083147,-0.011764,0.002394,-0.023920,-0.026703,-0.0639,-0.0149,0.0290,0.0211,-0.0047,0.0034
202504,-0.021080,0.016902,-0.037943,-0.138577,0.001144,-0.001013,0.001959,0.000634,-0.013142,-0.024308,-0.0084,-0.0186,-0.0340,-0.0285,-0.0267,0.0035
202505,0.045119,0.099728,-0.055741,0.012795,0.088401,0.083849,0.012220,0.038276,0.010412,0.029205,0.0606,-0.0072,-0.0288,0.0126,0.0251,0.0038
